In [1]:
import random
import datetime
from dateutil.relativedelta import relativedelta
from faker import Faker
random.seed(49)
fake = Faker(seed=49,locale="es")

In [113]:
formato_fecha = "%d-%m-%Y"

def str_a_date(fecha_str):
    d,m,y = fecha_str.split("-")
    return datetime.date(int(y), int(m), int(d))

def es_fecha_ultimos_anios(fecha, anios=2):
    hoy = datetime.datetime.today()
    fecha = str_a_date(fecha)
    anios_desde_fecha = relativedelta(hoy,fecha).years
    return anios_desde_fecha < anios

def get_fecha_ultimos_anios(anios=3, formato=formato_fecha): 
    return fake.date_between("-{}y".format(anios)).strftime(formato)

def get_precio(fecha, producto):
    """
        En base a una fecha y un codigo de producto, devuelve un precio simulando una inflacion random.
    """
    fecha_int = int((str_a_date(fecha)-datetime.date(2000,1,1)).total_seconds() / 1000000)
    return fecha_int * int(producto)

year = lambda x: x.split("-")[-1]

1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,fecha). Se pide resolver utilizando PySpark:

A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón,promedio).

C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

In [3]:
cantidad_alumnos = 20
padrones = [ x for x in range(1, cantidad_alumnos+1) ]
materias = ("ALGEBRA I", "ANALISIS I", "ALGEBRA II", "ANALISIS II", "PROBABILIDAD", "ALGORITMOS I", "ALGORITMOS II")
notas = list(range(1,11))
fechas = ("01-02-2019", "01-07-2019", "01-02-2018", "01-07-2018", "01-12-2018", "01-02-2017", "01-07-2017", "01-12-2017")
def alumno_random(): return tuple([random.choice(x) for x in (padrones, materias, notas, fechas)])

cantidad_registros = 50
notas = { alumno_random() for a in range(cantidad_registros) }

rdd_notas = sc.parallelize(notas)

es_aprobado = lambda x: x > 3

ejercicio_a = rdd_notas\
    .filter(lambda x: es_aprobado(x[2]))\
    .filter(lambda x: es_fecha_ultimos_anios(x[3]))\
    .map(lambda x: (x[0], 0))\
    .groupByKey()\
    .count()

print("{} alumnos de {} aprobaron al menos 1 materia en los últimos 2 años".format(ejercicio_a, cantidad_alumnos))

ejercicio_b = rdd_notas\
    .map(lambda x: (x[0], (x[2], 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+1))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))
# pide el RDD, no ejecuto ninguna acción
print(ejercicio_b)

nombres = [(x, fake.first_name(),fake.last_name()) for x in padrones]
rdd_nombres = sc.parallelize(nombres)

padron_mejor_promedio = ejercicio_b.max(lambda x: x[1])[0]
ejercicio_c = rdd_nombres.filter(lambda x: x[0]==padron_mejor_promedio).collect()[0]
print("'{}, {}' (padron {}) es el alumno con mejor promedio".format(ejercicio_c[2], ejercicio_c[1], ejercicio_c[0]))

15 alumnos de 20 aprobaron al menos 1 materia en los últimos 2 años
PythonRDD[10] at RDD at PythonRDD.scala:53
'Martin, Nuria' (padron 19) es el alumno con mejor promedio


---

2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de  producto, precio de venta) y en otro RDD detalle de los productos con (código de producto, descripción del producto, categoría). Se pide resolver utilizando PySpark:

A. Cuál es el producto más vendido.

B. Cuál es la categoría de productos más vendida.

C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)

D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.

E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

In [114]:
cantidad_ventas = 20000
ids_productos = list(range(1, 11))
categorias = ["Autos", "Motos", "Lanchas", "Aviones"]
productos = [ (a,fake.street_name(),random.choice(categorias)) for a in ids_productos ]

def crear_venta():
    fecha, producto = get_fecha_ultimos_anios(), random.choice(ids_productos)
    return (fecha, producto, get_precio(fecha, producto))

ventas = [crear_venta() for x in range(cantidad_ventas)]
rdd_ventas = sc.parallelize(ventas)
rdd_productos = sc.parallelize(productos)

In [115]:
# A. Cuál es el producto más vendido.
codigo_mas_vendido = rdd_ventas.map(lambda x: (x[1], 0))\
    .reduceByKey(lambda x,y: x+1)\
    .max(lambda x: x[1])[0]
producto_mas_vendido = rdd_productos.filter(lambda x: x[0]==codigo_mas_vendido).collect()[0]
print("El producto más vendido es {}".format(producto_mas_vendido))

El producto más vendido es (1, 'Alameda Emilio Noguera', 'Motos')


In [116]:
# B. Cuál es la categoría de productos más vendida.
categoria_mas_vendida = rdd_ventas.map(lambda x: (x[1], 0))\
    .join(rdd_productos.map(lambda x: (x[0], x[2])))\
    .map(lambda x: (x[1][1],0))\
    .reduceByKey(lambda x,y: x+1)\
    .max(lambda x: x[1])
print("La categoria mas vendida es {}, con {} ventas".format(*categoria_mas_vendida))

La categoria mas vendida es Autos, con 2021 ventas


In [117]:
# C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)
top_5 = rdd_ventas.map(lambda x: (x[1], 0))\
    .reduceByKey(lambda x,y: x+1)\
    .top(5, lambda x: x[1])
top_5 = sc.parallelize(top_5).join(rdd_productos.map(lambda x: (x[0],x[1]))).map(lambda x: (x[0], *x[1])).collect()
sorted(top_5, key=lambda x: x[1], reverse=True)

[(1, 229, 'Alameda Emilio Noguera'),
 (8, 225, 'C. de Juan Francisco Aramburu'),
 (3, 218, 'Plaza de Sara Barceló'),
 (9, 212, 'Paseo de Jose Antonio Pareja'),
 (6, 210, 'Ronda de Josep Español')]

In [118]:
# D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.

ventas_ultimo_anio = rdd_ventas.filter(lambda x: es_fecha_ultimos_anios(x[0], 1)) #(fecha de venta, código de producto, precio de venta)

def min_max_ventas(x, y):
    """ no fue magia """
    fecha_min_x, venta_min_x, fecha_max_x, venta_max_x, cant_x = x
    fecha_min_y, venta_min_y, fecha_max_y, venta_max_y, cant_y = y

    min_x = str_a_date(fecha_min_x) < str_a_date(fecha_min_y)
    fecha_min = fecha_min_x if min_x else fecha_min_y
    venta_min = venta_min_x if min_x else venta_min_y
    
    max_x = str_a_date(fecha_max_x) > str_a_date(fecha_max_y)
    fecha_max = fecha_max_x if max_x else fecha_max_y
    venta_max = venta_max_x if max_x else venta_max_y
        
    return (fecha_min, venta_min, fecha_max, venta_max, cant_x+1)

codigo, aumento = ventas_ultimo_anio.map(lambda x: (x[1], (x[0],x[2],x[0],x[2],0)))\
    .reduceByKey(min_max_ventas)\
    .filter(lambda x: int(x[1][-1]) > 50)\
    .map(lambda x: (x[0],int(x[1][3])-int(x[1][1])))\
    .max(lambda x: x[1])
producto = rdd_productos.filter(lambda x: x[0]==codigo).collect()[0]
print("El producto {} registró un aumento de {} el último año".format(producto, aumento))

El producto (10, 'Alameda Ivan Velázquez', 'Aviones') registró un aumento de 310 el último año


In [125]:
# D. Cuál es la categoria que registró mayor aumento de precio en el último año.
# Voy a considerar promedio año actual vs promedio año anterior

este_anio = datetime.date.today().year
anios = { str(este_anio-1), str(este_anio) }

actual = lambda x:  x==str(este_anio)

def mapper(x):
    """
        (cod, ((anio,precio),categoria)) -> (categoria, (suma_actual, suma_anterior, cant_actual, cant_anterior))
    """
    
    cod, ((anio, precio), categoria) = x
    es_actual = actual(anio)
    return (categoria, (precio if es_actual else 0, precio if not es_actual else 0, int(es_actual), int(not es_actual)))

def reducer(x,y):
    return tuple([sum(a) for a in zip(x,y)])

def map_dif(x):
    """
        (categoria, (suma_actual, suma_anterior, cant_actual, cant_anterior)) -> (categoria, aumento)
    """
    categoria, (suma_actual, suma_anterior, cant_actual, cant_anterior) = x
    avg_actual, avg_anterior = suma_actual/cant_actual , suma_anterior/cant_anterior
    aumento = (avg_actual/avg_anterior - 1) * 100
    return categoria, round(aumento,2)

aumentos = rdd_ventas.filter(lambda x: year(x[0]) in anios) \
    .map(lambda x: (x[1], (year(x[0]), x[2])))\
    .join(rdd_productos.map(lambda x: (x[0], x[2])))\
    .map(mapper)\
    .reduceByKey(reducer)\
    .map(map_dif)

max_aumento = aumentos.max(lambda x: x[1])

print("La categoria con más aumento ({}%) es '{}'".format(max_aumento[1], max_aumento[0]))

La categoria con más aumento (5.34%) es 'Lanchas'
